In [1]:
import os
upload_dir = "uploaded_docs"
os.makedirs(upload_dir, exist_ok=True)
print(f"請將你的 .txt, .pdf, .docx 胡志明檔案放到這個資料夾中： {upload_dir}")

upload_dir2 = "uploaded_docs2"
os.makedirs(upload_dir2, exist_ok=True)
print(f"請將你的 .txt, .pdf, .docx 河內檔案放到這個資料夾中： {upload_dir2}")

請將你的 .txt, .pdf, .docx 胡志明檔案放到這個資料夾中： uploaded_docs
請將你的 .txt, .pdf, .docx 河內檔案放到這個資料夾中： uploaded_docs2


In [1]:
# 執行後 Colab 會自動重啟 kernel（請在重啟後繼續執行下一個 cell）
import sys, subprocess, time, os, json

pkgs = [
    "langchain",
    "langchain-community",
    "faiss-cpu",
    "sentence-transformers",
    "transformers",
    "huggingface-hub",
    "python-docx",
    "pypdf",
    "unstructured",
    "gdown",
    "gradio",
    "openai"
]

print("開始安裝/升級套件（可能花一些時間）...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-U"] + pkgs)

print("安裝完成，將自動重啟執行階段以讓新安裝套件生效。")
# 自動重啟 kernel（Colab）
try:
    import IPython
    IPython.Application.instance().kernel.do_shutdown(restart=True)
except Exception as e:
    print("自動重啟失敗，請手動從選單點 Runtime -> Restart runtime。錯誤：", e)


開始安裝/升級套件（可能花一些時間）...
安裝完成，將自動重啟執行階段以讓新安裝套件生效。


In [1]:
# 重啟後執行：檢查版本並嘗試匯入需要的模組
import importlib, sys

def try_imports():
    try:
        import langchain
        import langchain_community
        from langchain.text_splitter import RecursiveCharacterTextSplitter
        from langchain_community.document_loaders import TextLoader, PyPDFLoader, UnstructuredWordDocumentLoader
        from langchain.embeddings import HuggingFaceEmbeddings
        from langchain.vectorstores import FAISS
        print("✅ 匯入成功：langchain 及相關模組可用")
        print("langchain 版本:", langchain.__version__)
        return True
    except Exception as e:
        print("❌ 匯入失敗：", repr(e))
        # 印出目前已安裝的相關套件版本（協助偵錯）
        import pkgutil
        def show_ver(name):
            try:
                m = importlib.import_module(name)
                return getattr(m, "__version__", "unknown")
            except Exception:
                return None
        print("已安裝版本快查：")
        names = ["langchain", "langchain_community", "faiss", "faiss_cpu", "sentence_transformers", "transformers"]
        for n in names:
            v = show_ver(n)
            print(f"  {n}: {v}")
        return False

ok = try_imports()
if not ok:
    print("\n如果仍然失敗，請執行下一格（fallback 解法），它不依賴 langchain.text_splitter。")


❌ 匯入失敗： ModuleNotFoundError("No module named 'langchain.text_splitter'")
已安裝版本快查：
  langchain: 1.2.0
  langchain_community: 0.4.1
  faiss: 1.13.2
  faiss_cpu: None


  sentence_transformers: 5.2.0
  transformers: 4.57.3

如果仍然失敗，請執行下一格（fallback 解法），它不依賴 langchain.text_splitter。


In [2]:
# 在 Colab 執行：安裝缺少套件並建立 faiss_db.zip（支援 pdfminer 或 pypdf）
import sys, subprocess, os, shutil, json
from pathlib import Path

def pip_install(packages):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-U"] + packages)

print("安裝/更新必要套件：pdfminer.six, python-docx, pypdf, sentence-transformers, faiss-cpu")
pip_install(["pdfminer.six", "python-docx", "pypdf", "sentence-transformers", "faiss-cpu", "tqdm"])

# 下面開始載入模組並執行 fallback 流程
print("載入模組...")
import docx
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
from tqdm import tqdm

# 嘗試匯入 pdfminer，若失敗則 later 使用 pypdf 做 fallback
use_pdfminer = True
try:
    from pdfminer.high_level import extract_text as pdf_extract_text
    print("使用 pdfminer 解析 PDF。")
except Exception as e:
    print("無法匯入 pdfminer，將改用 pypdf 作為 fallback。錯誤：", e)
    use_pdfminer = False
    # 設定 pdf_extract_text 使用 pypdf
    try:
        from pypdf import PdfReader
        def pdf_extract_text(path):
            text_pages = []
            r = PdfReader(path)
            for p in r.pages:
                try:
                    t = p.extract_text() or ""
                except Exception:
                    t = ""
                text_pages.append(t)
            return "\n".join(text_pages)
        print("已設定 pypdf 作為 PDF 解析 fallback。")
    except Exception as e2:
        print("pypdf 也無法使用，PDF 解析將失敗。錯誤：", e2)
        # 仍可處理 txt/docx

# 設定目錄
UPLOAD_DIR = "uploaded_docs"
OUT_DIR = "faiss_db"
ZIP_NAME = "faiss_db.zip"
os.makedirs(UPLOAD_DIR, exist_ok=True)

# Loader functions
def load_txt(path):
    try:
        with open(path, "r", encoding="utf-8", errors="ignore") as f:
            return f.read()
    except Exception:
        with open(path, "r", encoding="latin-1", errors="ignore") as f:
            return f.read()

def load_docx(path):
    try:
        doc = docx.Document(path)
        return "\n".join(p.text for p in doc.paragraphs)
    except Exception as e:
        print(f"docx 解析失敗 ({path})：", e)
        return ""

def load_pdf_safe(path):
    try:
        return pdf_extract_text(path)
    except Exception as e:
        print(f"PDF 解析失敗 ({path})：", e)
        return ""

def load_all_docs(folder=UPLOAD_DIR):
    docs = []
    for fname in sorted(os.listdir(folder)):
        path = os.path.join(folder, fname)
        if fname.lower().endswith(".txt"):
            text = load_txt(path)
        elif fname.lower().endswith(".pdf"):
            text = load_pdf_safe(path)
        elif fname.lower().endswith(".docx"):
            text = load_docx(path)
        else:
            print("跳過不支援檔案：", fname)
            continue
        print(f"已載入: {fname} (長度 {len(text)} 字元)")
        docs.append({"source": fname, "text": text})
    return docs

# 簡單分段器
def simple_chunk_text(text, chunk_size=500, overlap=100):
    chunks = []
    if not text:
        return chunks
    start = 0
    L = len(text)
    while start < L:
        end = min(L, start + chunk_size)
        chunk = text[start:end]
        chunks.append(chunk)
        if end == L:
            break
        start = max(0, end - overlap)
    return chunks

# 建立 FAISS index
def build_faiss_from_docs(docs, model_name="sentence-transformers/all-MiniLM-L6-v2", chunk_size=500, overlap=100):
    print("載入 embedding 模型：", model_name)
    model = SentenceTransformer(model_name)
    chunk_texts = []
    metadata = []
    for d in docs:
        chunks = simple_chunk_text(d["text"], chunk_size=chunk_size, overlap=overlap)
        for i, c in enumerate(chunks):
            chunk_texts.append(c)
            metadata.append({"source": d["source"], "chunk_id": i})
    total_chunks = len(chunk_texts)
    print("總 chunk 數：", total_chunks)
    if total_chunks == 0:
        raise SystemExit("沒有可用 chunk，請確認 uploaded_docs 是否包含檔案並成功解析。")

    print("開始做 embedding（分批處理）...")
    # 為避免一次吃太多記憶體，分批 encode
    batch_size = 64
    embeddings = []
    for i in tqdm(range(0, total_chunks, batch_size)):
        batch_texts = chunk_texts[i:i+batch_size]
        emb = model.encode(batch_texts, show_progress_bar=False, convert_to_numpy=True)
        embeddings.append(emb)
    embeddings = np.vstack(embeddings)
    print("embedding shape:", embeddings.shape)

    # normalize to unit vectors for cosine via inner product
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
    norms[norms==0] = 1.0
    embeddings_norm = embeddings / norms

    dim = embeddings_norm.shape[1]
    # 建立 IndexFlatIP（內積，相當於 cosine if vectors normalized）
    index = faiss.IndexFlatIP(dim)
    index.add(embeddings_norm.astype("float32"))
    print("FAISS index 建立完成，總向量數：", index.ntotal)

    # 儲存 index 與 chunks & metadata
    if os.path.exists(OUT_DIR):
        shutil.rmtree(OUT_DIR)
    os.makedirs(OUT_DIR, exist_ok=True)
    faiss.write_index(index, os.path.join(OUT_DIR, "index.faiss"))
    with open(os.path.join(OUT_DIR, "chunks.json"), "w", encoding="utf-8") as f:
        json.dump({"texts": chunk_texts, "metadata": metadata}, f, ensure_ascii=False, indent=2)
    print("已儲存 index 與 chunks.json 到資料夾：", OUT_DIR)

    # 壓縮
    if os.path.exists(ZIP_NAME):
        os.remove(ZIP_NAME)
    shutil.make_archive(base_name=ZIP_NAME.replace(".zip",""), format="zip", root_dir=OUT_DIR)
    print("✅ 已壓縮為", ZIP_NAME)

# 主流程
docs = load_all_docs(UPLOAD_DIR)
build_faiss_from_docs(docs)
print("完成：faiss_db.zip 已建立，你可以在 Colab 左側看到該檔案。")


安裝/更新必要套件：pdfminer.six, python-docx, pypdf, sentence-transformers, faiss-cpu
載入模組...
使用 pdfminer 解析 PDF。
已載入: HoChiMinh.docx (長度 10183 字元)
載入 embedding 模型： sentence-transformers/all-MiniLM-L6-v2


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

總 chunk 數： 26
開始做 embedding（分批處理）...


100%|██████████| 1/1 [00:03<00:00,  3.92s/it]

embedding shape: (26, 384)
FAISS index 建立完成，總向量數： 26
已儲存 index 與 chunks.json 到資料夾： faiss_db
✅ 已壓縮為 faiss_db.zip
完成：faiss_db.zip 已建立，你可以在 Colab 左側看到該檔案。


In [3]:
# 在 Colab 執行：安裝缺少套件並建立 faiss_db2.zip（支援 pdfminer 或 pypdf）
import sys, subprocess, os, shutil, json
from pathlib import Path

def pip_install(packages):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-U"] + packages)

print("安裝/更新必要套件：pdfminer.six, python-docx, pypdf, sentence-transformers, faiss-cpu")
pip_install(["pdfminer.six", "python-docx", "pypdf", "sentence-transformers", "faiss-cpu", "tqdm"])

# 下面開始載入模組並執行 fallback 流程
print("載入模組...")
import docx
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
from tqdm import tqdm

# 嘗試匯入 pdfminer，若失敗則 later 使用 pypdf 做 fallback
use_pdfminer = True
try:
    from pdfminer.high_level import extract_text as pdf_extract_text
    print("使用 pdfminer 解析 PDF。")
except Exception as e:
    print("無法匯入 pdfminer，將改用 pypdf 作為 fallback。錯誤：", e)
    use_pdfminer = False
    # 設定 pdf_extract_text 使用 pypdf
    try:
        from pypdf import PdfReader
        def pdf_extract_text(path):
            text_pages = []
            r = PdfReader(path)
            for p in r.pages:
                try:
                    t = p.extract_text() or ""
                except Exception:
                    t = ""
                text_pages.append(t)
            return "\n".join(text_pages)
        print("已設定 pypdf 作為 PDF 解析 fallback。")
    except Exception as e2:
        print("pypdf 也無法使用，PDF 解析將失敗。錯誤：", e2)
        # 仍可處理 txt/docx

# 設定目錄（已改為 uploaded_docs2 / faiss_db2）
UPLOAD_DIR = "uploaded_docs2"
OUT_DIR = "faiss_db2"
ZIP_NAME = "faiss_db2.zip"
os.makedirs(UPLOAD_DIR, exist_ok=True)

# Loader functions
def load_txt(path):
    try:
        with open(path, "r", encoding="utf-8", errors="ignore") as f:
            return f.read()
    except Exception:
        with open(path, "r", encoding="latin-1", errors="ignore") as f:
            return f.read()

def load_docx(path):
    try:
        doc = docx.Document(path)
        return "\n".join(p.text for p in doc.paragraphs)
    except Exception as e:
        print(f"docx 解析失敗 ({path})：", e)
        return ""

def load_pdf_safe(path):
    try:
        return pdf_extract_text(path)
    except Exception as e:
        print(f"PDF 解析失敗 ({path})：", e)
        return ""

def load_all_docs(folder=UPLOAD_DIR):
    docs = []
    for fname in sorted(os.listdir(folder)):
        path = os.path.join(folder, fname)
        if fname.lower().endswith(".txt"):
            text = load_txt(path)
        elif fname.lower().endswith(".pdf"):
            text = load_pdf_safe(path)
        elif fname.lower().endswith(".docx"):
            text = load_docx(path)
        else:
            print("跳過不支援檔案：", fname)
            continue
        print(f"已載入: {fname} (長度 {len(text)} 字元)")
        docs.append({"source": fname, "text": text})
    return docs

# 簡單分段器
def simple_chunk_text(text, chunk_size=500, overlap=100):
    chunks = []
    if not text:
        return chunks
    start = 0
    L = len(text)
    while start < L:
        end = min(L, start + chunk_size)
        chunk = text[start:end]
        chunks.append(chunk)
        if end == L:
            break
        start = max(0, end - overlap)
    return chunks

# 建立 FAISS index
def build_faiss_from_docs(docs, model_name="sentence-transformers/all-MiniLM-L6-v2", chunk_size=500, overlap=100):
    print("載入 embedding 模型：", model_name)
    model = SentenceTransformer(model_name)
    chunk_texts = []
    metadata = []
    for d in docs:
        chunks = simple_chunk_text(d["text"], chunk_size=chunk_size, overlap=overlap)
        for i, c in enumerate(chunks):
            chunk_texts.append(c)
            metadata.append({"source": d["source"], "chunk_id": i})
    total_chunks = len(chunk_texts)
    print("總 chunk 數：", total_chunks)
    if total_chunks == 0:
        raise SystemExit(f"沒有可用 chunk，請確認 {UPLOAD_DIR} 是否包含檔案並成功解析。")

    print("開始做 embedding（分批處理）...")
    # 為避免一次吃太多記憶體，分批 encode
    batch_size = 64
    embeddings = []
    for i in tqdm(range(0, total_chunks, batch_size)):
        batch_texts = chunk_texts[i:i+batch_size]
        emb = model.encode(batch_texts, show_progress_bar=False, convert_to_numpy=True)
        embeddings.append(emb)
    embeddings = np.vstack(embeddings)
    print("embedding shape:", embeddings.shape)

    # normalize to unit vectors for cosine via inner product
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
    norms[norms==0] = 1.0
    embeddings_norm = embeddings / norms

    dim = embeddings_norm.shape[1]
    # 建立 IndexFlatIP（內積，相當於 cosine if vectors normalized）
    index = faiss.IndexFlatIP(dim)
    index.add(embeddings_norm.astype("float32"))
    print("FAISS index 建立完成，總向量數：", index.ntotal)

    # 儲存 index 與 chunks & metadata
    if os.path.exists(OUT_DIR):
        shutil.rmtree(OUT_DIR)
    os.makedirs(OUT_DIR, exist_ok=True)
    faiss.write_index(index, os.path.join(OUT_DIR, "index.faiss"))
    with open(os.path.join(OUT_DIR, "chunks.json"), "w", encoding="utf-8") as f:
        json.dump({"texts": chunk_texts, "metadata": metadata}, f, ensure_ascii=False, indent=2)
    print("已儲存 index 與 chunks.json 到資料夾：", OUT_DIR)

    # 壓縮
    if os.path.exists(ZIP_NAME):
        os.remove(ZIP_NAME)
    shutil.make_archive(base_name=ZIP_NAME.replace(".zip",""), format="zip", root_dir=OUT_DIR)
    print("✅ 已壓縮為", ZIP_NAME)

# 主流程
docs = load_all_docs(UPLOAD_DIR)
build_faiss_from_docs(docs)
print("完成：faiss_db2.zip 已建立，你可以在 Colab 左側看到該檔案。")


安裝/更新必要套件：pdfminer.six, python-docx, pypdf, sentence-transformers, faiss-cpu
載入模組...
使用 pdfminer 解析 PDF。
已載入: Hanoi.docx (長度 9734 字元)
載入 embedding 模型： sentence-transformers/all-MiniLM-L6-v2
總 chunk 數： 25
開始做 embedding（分批處理）...


100%|██████████| 1/1 [00:02<00:00,  2.70s/it]

embedding shape: (25, 384)
FAISS index 建立完成，總向量數： 25
已儲存 index 與 chunks.json 到資料夾： faiss_db2
✅ 已壓縮為 faiss_db2.zip
完成：faiss_db2.zip 已建立，你可以在 Colab 左側看到該檔案。


In [5]:
# 實際版本


import os
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import requests
from google.colab import userdata
import gradio as gr

# 載入 embedding 模型（共用）
print("載入 embedding 模型: sentence-transformers/all-MiniLM-L6-v2")
embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# 預先載入兩個 FAISS DB（可改為 lazy load）
def load_faiss_dir(dir_path):
    idx = faiss.read_index(os.path.join(dir_path, "index.faiss"))
    with open(os.path.join(dir_path, "chunks.json"), "r", encoding="utf-8") as f:
        chunk_data = json.load(f)
    return {
        "index": idx,
        "texts": chunk_data["texts"],
        "meta": chunk_data["metadata"]
    }

print("載入 faiss_db ...")
faiss_db = load_faiss_dir("faiss_db")

print("載入 faiss_db2 ...")
faiss_db2 = load_faiss_dir("faiss_db2")

# 選擇要使用的 FAISS 資料庫
def select_faiss(query):
    q_lower = query.lower()
    has_hcm = "胡志明" in q_lower or "胡志明市" in q_lower or "hcm" in q_lower
    has_hanoi = "河內" in q_lower or "河內市" in q_lower or "hanoi" in q_lower

    # 若同時包含或都不包含，使用 faiss_db
    if has_hcm and not has_hanoi:
        return faiss_db
    if has_hanoi and not has_hcm:
        return faiss_db2
    return faiss_db

# retrieve_chunks 改為接受 index 與 chunks
def retrieve_chunks_from(index_obj, chunk_texts, chunk_meta, query, top_k=5):
    q_emb = embed_model.encode([query], convert_to_numpy=True)
    q_emb = q_emb / np.linalg.norm(q_emb, axis=1, keepdims=True)

    scores, indices = index_obj.search(q_emb.astype("float32"), top_k)

    results = []
    for idx, score in zip(indices[0], scores[0]):
        results.append({
            "text": chunk_texts[idx],
            "meta": chunk_meta[idx],
            "score": float(score)
        })
    return results

# build_rag_prompt 與 ollama_generate 保持不變
def build_rag_prompt(question, retrieved):
    context = "\n\n".join(
        f"[來源: {r['meta']['source']}]\n{r['text']}"
        for r in retrieved
    )
    prompt = f"""你是一個嚴謹的助教，請根據【資料】回答【問題】。
若資料中沒有答案，請明確回答「資料不足，無法回答」。

【資料】
{context}

【問題】
{question}

請用繁體中文回答。
"""
    return prompt

API_KEY = userdata.get("ollama_key")
OLLAMA_URL = "https://api-gateway.netdb.csie.ncku.edu.tw/api/generate"

def ollama_generate(prompt, model="gpt-oss:120b", temperature=0.2):
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json",
    }
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False,
        "options": {"temperature": temperature}
    }
    r = requests.post(OLLAMA_URL, headers=headers, json=payload, timeout=180)
    r.raise_for_status()
    return r.json()["response"]

# rag_chat 先選擇資料庫，再檢索
def rag_chat(query, top_k=5):
    db = select_faiss(query)
    retrieved = retrieve_chunks_from(db["index"], db["texts"], db["meta"], query, top_k=top_k)
    prompt = build_rag_prompt(query, retrieved)
    answer = ollama_generate(prompt)

    sources = "\n".join(
        f"- {r['meta']['source']} (score={r['score']:.3f})"
        for r in retrieved
    )
    return answer, sources

# Gradio UI 保持不變
with gr.Blocks(title="RAG + Ollama (FAISS)") as demo:
    gr.Markdown("## 📚 RAG 問答系統（FAISS + Ollama）")
    query = gr.Textbox(label="輸入問題", placeholder="例如：請給我胡志明旅遊攻略", lines=2)
    top_k = gr.Slider(minimum=1, maximum=10, value=5, step=1, label="檢索 chunk 數量 (top-k)")
    btn = gr.Button("送出查詢")
    answer = gr.Textbox(label="模型回答", lines=10)
    sources = gr.Textbox(label="使用到的資料來源", lines=6)
    btn.click(fn=rag_chat, inputs=[query, top_k], outputs=[answer, sources])

demo.launch(debug=True)


載入 embedding 模型: sentence-transformers/all-MiniLM-L6-v2
載入 faiss_db ...
載入 faiss_db2 ...
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://9319aa2439261e969d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://9319aa2439261e969d.gradio.live
